# Set Up

In [14]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

In [15]:
# reading excel file for outages 
circuit_outage= pd.read_excel("Data/circuit_outages_ibec.xlsx", sheet_name = "Circuit Outage", usecols= "A:G") # circuit outage sheet 
circuit_lookup = pd.read_excel("Data/circuit_outages_ibec.xlsx", sheet_name = "LookUp", usecols= "A:J") # lookup sheet - circuits
outage_cause_lookup = pd.read_excel("Data/circuit_outages_ibec.xlsx", sheet_name = "LookUp", usecols= "L:M", nrows=8) # lookup sheet - outage causes

# Data Enrichment

Renaming columns for intuitive interpretation.

In [16]:
# renaming outage dataframe
circuit_outage = circuit_outage.rename(columns = {'Circuit Number':'circuit_number', 
                                                  'Circuit Name':'circuit_name', 
                                                  'Outage Date':'outage_date', 
                                                  'Outage CauseCode':'outage_cause_code',
                                                  'Outage Cause':'outage_cause', 
                                                  'Outage Duration (min)':'outage_duration', 
                                                  'Customers Affected':'customers_affected'})

In [17]:
# renaming circuit dataframe
circuit_lookup = circuit_lookup.rename(columns={'Circuit Number':'circuit_number', 
                                  'Circuit Name':'circuit_name', 
                                  'KV':'voltage', 
                                  'Region':'region', 
                                  'Customer Count':'customer_count',
                                  'Circuit Miles':'circuit_miles',
                                  '% Overhead':'percent_overhead',
                                  '% Underground':'percent_underground',
                                  '2023 SAIDI':'saidi_2023',
                                  '2023 SAIFI':'saifi_2023'}) # column labels for circuit table

Enriching Data with new features to prepare for analysis.

Circuit Outage Table Features
* **Season** - season of the year outage occured in.
    - To identify how outages are related to seasonal changes.
* **CMI** - customer minutes interrupted per **outage**
    - Calculated as: '# customers affected x duration of outage'

In [18]:
# functioon to create season column from date column 
def get_season(date):
    month = date.month
    if month in [12,1,2]:
        return 'Winter'
    elif month in [3,4,5]:
        return 'Spring'
    elif month in [6,7,8]:
        return 'Summer'
    elif month in [9,10,11]:
        return 'Fall'
        
circuit_outage['season'] = circuit_outage['outage_date'].apply(get_season) # return season outage occured in
circuit_outage['cmi'] = circuit_outage['customers_affected'] * circuit_outage['outage_duration'] # return total customer minutes interrupted of




Circuit Lookup Table Features<br>
2024 Circuit Reliability Metrics - Indices used to understand how a circuit is performing.
* 2024 SAIDI = 'total outage time / customers served'
* 2024 SAIFI = 'total outages / customers served'

Change From Previous Year - to examine which circuits are experiencing the greatest increase/decrease in outages. 
* SAIDI change from previous year = saidi_2024 - saidi_2023
* SAIFI change from previous year = saifi_2024 - saidi_2023

**2024 CMI** - total customer minutes interrupted **per circuit for the year 2024**

In [21]:
# circuit table features 

# SAIDI 2024
total_outage_time_2024 = circuit_outage.groupby('circuit_name')['outage_duration'].sum().reset_index(name='total_outage_time_2024')
circuit_lookup = circuit_lookup.merge(total_outage_time_2024, on ='circuit_name', how = 'left')
circuit_lookup['saidi_2024'] = ( circuit_lookup['total_outage_time_2024']/circuit['customer_count'] ).round(2)
# SAIFI 2024
total_outages_2024 =  circuit_outage.groupby('circuit_name').size().reset_index(name = 'total_outages_2024')
circuit_lookup = circuit_lookup.merge(total_outages_2024, on ='circuit_name', how = 'left')
circuit_lookup['saifi_2024'] = ( circuit['total_outages_2024'] / circuit_lookup['customer_count'] ).round(3)

# Changes from Previous Year
circuit_lookup['saidi_diff_from_prev_year'] = circuit_lookup['saidi_2024'] - circuit_lookup['saidi_2023']
circuit_lookup['saifi_diff_from_prev_year'] = circuit_lookup['saifi_2024'] - circuit_lookup['saifi_2023']

# Create CMI Column Pe
cmi_per_circuit = circuit_outage.groupby("circuit_name")["cmi"].sum().reset_index()
circuit_lookup = circuit_lookup.merge(cmi_per_circuit, on="circuit_name", how="left")
circuit_lookup = circuit_lookup.rename(columns={"cmi": "cmi_2024"})


KeyError: 'total_outage_time_2024'

Join enriched dataframes to denormalize data 

In [ ]:

df = (circuit_outage.merge(circuit, on=['circuit_number','circuit_name'], how='inner')
)


In [ ]:
circuit.head()


In [ ]:
circuit.columns
new_column_order = ['circuit_number', 'circuit_name', 'voltage', 'region', 'customer_count',
       'circuit_miles', 'percent_overhead', 'percent_underground',
       'saidi_2023','saidi_2024', 'saidi_diff_from_prev_year',
        'saifi_2023','saifi_2024','saifi_diff_from_prev_year','total_outage_time_2024',  'total_outages_2024']
circuit = circuit[new_column_order]
circuit.head(20)

In [ ]:
df.columns

In [ ]:
new_column_order = ['circuit_number', 'circuit_name', 'outage_date', 'season', 'outage_cause_code',
       'outage_cause', 'outage_duration', 'customers_affected',
       'cmi', 'voltage', 'region', 'customer_count', 'circuit_miles',
       'percent_overhead', 'percent_underground', 'saidi_2023','saidi_2024', 'saidi_diff_from_prev_year',
        'saifi_2023','saifi_2024','saifi_diff_from_prev_year','total_outage_time_2024',  'total_outages_2024'
         ]
df = df[new_column_order]

In [ ]:
df.head(20)

In [ ]:
circuit.head(20)

In [ ]:
# renaming outage cause 
outage_cause = outage_cause.rename(columns={'Outage Cause Code':'outage_cause_code', 
                                            'Outage Cause':'outage_cause'}) # column labels for outage cause

In [ ]:
# Circuits Ordered by total Customer Minutes Interrupted

total_cmi = (
    df
    .groupby(['circuit_name', 'region', 'voltage'])['cmi']
    .sum()
    .reset_index(name='total_cmi')  
    .sort_values(by='total_cmi', ascending=False)
    .reset_index(drop=True)
)


total_cmi